## Import Modules 

In [1]:
# Import the dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Loading the Dataset

In [2]:
# Read the dataset
adress = r'Spam Email raw text for NLP.csv'
df = pd.read_csv(adress)

In [3]:
# First five rows
df.head()

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6


## PreProcessing

In [4]:
# Check for nulls
df.isnull().sum()

CATEGORY     0
MESSAGE      0
FILE_NAME    0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CATEGORY   5796 non-null   int64 
 1   MESSAGE    5796 non-null   object
 2   FILE_NAME  5796 non-null   object
dtypes: int64(1), object(2)
memory usage: 136.0+ KB


In [10]:
# Function for standardizing lenght of sequences
def get_sequences(texts, tokenizer, train = True, max_seq_length = None):

    sequences = tokenizer.texts_to_sequences(texts)

    if train == True:
        max_seq_length = np.max(list(map(lambda x : len(x), sequences)))

    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen = max_seq_length, padding = 'post')

    return sequences


In [11]:
# Define a function for preprocessing
def preprocess(df):
    df = df.copy()
    
    # Drop FILE_NAME column
    df = df.drop('FILE_NAME', axis = 1)
    
    # Split data into X and y
    y = df['CATEGORY']
    X = df['MESSAGE']

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 1)

    # Create tokenizer
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 30000)

    # Fit the tokenizer
    tokenizer.fit_on_texts(X_train)

    # Convert text to sequences
    X_train = get_sequences(X_train, tokenizer, train = True)
    X_test = get_sequences(X_test, tokenizer, train = False, max_seq_length = X_train.shape[1])
    
    return X_train, X_test, y_train, y_test

In [12]:
# Split data into train and test set
X_train, X_test, y_train, y_test = preprocess(df)

In [13]:
# Check for imbalance data
y_train.value_counts()

CATEGORY
0    2738
1    1319
Name: count, dtype: int64

## Train the model

In [14]:
X_train.shape

(4057, 14804)

In [17]:
inputs = tf.keras.Input(shape = (14804,))
embedding = tf.keras.layers.Embedding(
    input_dim = 30000,
    output_dim = 64
)(inputs)

In [19]:
flatten = tf.keras.layers.Flatten()(embedding)

In [21]:
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(flatten)

In [22]:
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [23]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        tf.keras.metrics.AUC(name = 'auc')
    ]
)

In [24]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14804)]           0         
                                                                 
 embedding (Embedding)       (None, 14804, 64)         1920000   
                                                                 
 flatten_1 (Flatten)         (None, 947456)            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 947457    
                                                                 
Total params: 2,867,457
Trainable params: 2,867,457
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
history = model.fit(
        X_train, 
        y_train, 
        validation_split = 0.2, 
        batch_size = 32, 
        epochs = 100, 
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor = 'val_loss', 
                patience = 3, 
                restore_best_weights = True
            )
        ]
)

Epoch 1/100
102/102 [==============================] - 22s 216ms/step - loss: 0.0330 - accuracy: 0.9929 - auc: 0.9997 - val_loss: 0.0393 - val_accuracy: 0.9877 - val_auc: 0.9991
Epoch 2/100
102/102 [==============================] - 22s 216ms/step - loss: 0.0113 - accuracy: 0.9991 - auc: 1.0000 - val_loss: 0.0350 - val_accuracy: 0.9877 - val_auc: 0.9991
Epoch 3/100
102/102 [==============================] - 23s 221ms/step - loss: 0.0062 - accuracy: 0.9994 - auc: 1.0000 - val_loss: 0.0336 - val_accuracy: 0.9877 - val_auc: 0.9991
Epoch 4/100
102/102 [==============================] - 22s 217ms/step - loss: 0.0041 - accuracy: 0.9994 - auc: 1.0000 - val_loss: 0.0322 - val_accuracy: 0.9901 - val_auc: 0.9991
Epoch 5/100
102/102 [==============================] - 22s 217ms/step - loss: 0.0030 - accuracy: 0.9994 - auc: 1.0000 - val_loss: 0.0329 - val_accuracy: 0.9877 - val_auc: 0.9991
Epoch 6/100
102/102 [==============================] - 22s 221ms/step - loss: 0.0022 - accuracy: 0.9994 - auc:

## Results

In [29]:
results = model.evaluate(X_test, y_test, verbose = 0)

print('Test Loss : {:.4f}'.format(results[0]))
print('Test Accuracy : {:.2f}'.format(results[1]*100))
print('Test AUC : {:.4f}'.format(results[2]))

Test Loss : 0.0240
Test Accuracy : 99.31
Test AUC : 0.9988
